In [1]:
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2DTranspose
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
SHAPE = (300, 300)
SHAPE2 = (300, 300, 1)

In [3]:
def add_noise(img):
    nf = 0.5
    norm = np.random.normal(loc=0.0, scale=1.0, size=img.shape)
    return img + nf + norm

In [4]:
grounddata = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, dtype='float32')
xtrain = grounddata.flow_from_directory('data/train', target_size=SHAPE, batch_size=10, color_mode='grayscale', class_mode='categorical')
xtest = grounddata.flow_from_directory('data/test', target_size=SHAPE, batch_size=10, color_mode='grayscale', class_mode='categorical')

Found 5436 images belonging to 36 classes.
Found 5436 images belonging to 36 classes.


In [5]:
noisydata = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, preprocessing_function=add_noise, dtype='float32')
xtrainnoisy = noisydata.flow_from_directory('data/train', target_size=(300, 300), batch_size=10, color_mode='grayscale', class_mode='categorical')
xtestnoisy = noisydata.flow_from_directory('data/test', target_size=(300, 300), batch_size=10, color_mode='grayscale', class_mode='categorical')

Found 5436 images belonging to 36 classes.
Found 5436 images belonging to 36 classes.


In [6]:
classifier = Sequential()
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=SHAPE2, strides=(2, 2), padding='same'))
classifier.add(Conv2D(16, kernel_size=(3, 3), activation='relu', strides=(2, 2), padding='same'))
classifier.add(Conv2D(8, kernel_size=(3, 3), activation='relu', strides=(2, 2), padding='same'))
classifier.add(Conv2DTranspose(8, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same'))
classifier.add(Conv2DTranspose(16, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same'))
classifier.add(Conv2DTranspose(32, kernel_size=(3,3), activation='relu', strides=(2,2), padding='same'))
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.50))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.50))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=36, activation='softmax'))

In [7]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        4624      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 8)         1160      
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 76, 76, 8)         584       
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 152, 152, 16)      1168      
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 304, 304, 32)      4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 302, 302, 32)      9

In [8]:
classifier.fit(xtrainnoisy,
               epochs=100,
               batch_size=128,
               validation_data=xtestnoisy,
               shuffle=True)

Epoch 1/100
544/544 [==============================] - 707s 1s/step - loss: 3.5854 - accuracy: 0.0223 - val_loss: 3.5836 - val_accuracy: 0.0278
Epoch 2/100
544/544 [==============================] - 745s 1s/step - loss: 3.5849 - accuracy: 0.0254 - val_loss: 3.5836 - val_accuracy: 0.0278
Epoch 3/100
544/544 [==============================] - 738s 1s/step - loss: 3.5848 - accuracy: 0.0241 - val_loss: 3.5835 - val_accuracy: 0.0278
Epoch 4/100
544/544 [==============================] - 890s 2s/step - loss: 3.5846 - accuracy: 0.0248 - val_loss: 3.5835 - val_accuracy: 0.0278
Epoch 5/100
544/544 [==============================] - 841s 2s/step - loss: 3.5844 - accuracy: 0.0235 - val_loss: 3.5835 - val_accuracy: 0.0278
Epoch 6/100
544/544 [==============================] - 723s 1s/step - loss: 3.5845 - accuracy: 0.0254 - val_loss: 3.5835 - val_accuracy: 0.0278
Epoch 7/100
544/544 [==============================] - 736s 1s/step - loss: 3.5845 - accuracy: 0.0213 - val_loss: 3.5835 - val_accuracy: